In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import tensorflow as tf
from collections import Counter

%matplotlib inline

/home/samuel/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Imports

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
race = "hispanic"
#race = "white"
#race = "mixed"

In [4]:
X = np.load('../Data/' + race + '/X.npy')
Y2 = np.load('../Data/' + race + '/Y2.npy')

In [5]:
# X_hispanic = np.load('../Data/hispanic/X.npy')
# Y_hispanic = np.load('../Data/hispanic/Y2.npy')
# X_white = np.load('../Data/white/X.npy')
# Y_white = np.load('../Data/white/Y2.npy')
#X_mixed = np.load('../Data/mixed/X.npy')
#Y_mixed = np.load('../Data/mixed/Y2.npy')

In [6]:
def split_train_test(X, Y):
    # shuffle
    np.random.seed(97)
    idx = np.random.permutation(len(X))
    X = X[idx]
    Y = Y[idx]
    
    #X = X[:size_hispanic]
    #Y = Y[:size_hispanic]

    # split into training and test sets
    TEST_SET_SIZE = int(0.1*len(Y))
    X_train, X_test = X[:-TEST_SET_SIZE], X[-TEST_SET_SIZE:]
    Y_train, Y_test = Y[:-TEST_SET_SIZE].astype(int), Y[-TEST_SET_SIZE:].astype(int)
    return X_train, X_test, Y_train, Y_test

In [7]:
X_train, X_test, Y_train, Y_test = split_train_test(X, Y2)

In [ ]:
#X_train_h, X_test_h, Y_train_h, Y_test_h = split_train_test(X_hispanic, Y_hispanic)
#X_train_w, X_test_w, Y_train_w, Y_test_w = split_train_test(X_white, Y_white)
#X_train_m, X_test_m, Y_train_m, Y_test_m = split_train_test(X_mixed, Y_mixed)

# Feature Scaling
Fit scaler based on training data, then transform both the training and test data.

In [8]:
# feature scaling: scale features based on training data only
from sklearn.preprocessing import StandardScaler, MinMaxScaler

def feature_scale(X_train, X_test):
    
    mm_scaler = MinMaxScaler(feature_range=(-1,1))
    X_train[:,-4:] = mm_scaler.fit_transform(X_train[:,-4:])
    X_test[:,-4:] = mm_scaler.transform(X_test[:,-4:])
    
    std_scaler = StandardScaler()
    X_train[:,:-4] = std_scaler.fit_transform(X_train[:,:-4])
    X_test[:,:-4] = std_scaler.transform(X_test[:,:-4])
    return X_train, X_test

X_train, X_test = feature_scale(X_train, X_test)
#X_train_h, X_test_h = feature_scale(X_train_h, X_test_h)
#X_train_w, X_test_w = feature_scale(X_train_w, X_test_w)
#X_train_m, X_test_m = feature_scale(X_train_m, X_test_m)

/home/samuel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/samuel/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


# Results Function

In [9]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix

def results(classifier):
    Y_pred_test = classifier.predict(X_test)
    print("Test accuracy score: " + str(accuracy_score(Y_test.astype(int), Y_pred_test)))
    print("ROC: " + str(roc_auc_score(Y_test, classifier.predict_proba(X_test)[:,1])))
    #print("ROC: " + str(roc_auc_score(Y_test, Y_pred_test)))
    matrix = confusion_matrix(Y_test.astype(int), Y_pred_test)
    tn, fp, fn, tp = matrix.ravel()
    ppv = tp/(tp+fp)
    npv = tn/(tn+fn)
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    g_mean = np.sqrt(sensitivity*specificity)
    print("PPV: " + str(ppv))
    print("NPV: " + str(npv))
    print("Sensitivity: " + str(sensitivity))
    print("Specificity: " + str(specificity))
    print("G-Mean: " + str(g_mean))
    print("Confusion matrix:\n" + str(matrix))
    

In [10]:
import pickle
with open('white_codes.pkl', 'rb') as f_w:
    white_codes = pickle.load(f_w)
with open('hispanic_codes.pkl', 'rb') as f_h:
    hispanic_codes = pickle.load(f_h)

In [11]:
def feature_importance_rank(array):
    codes = None
    if race == "white":
        codes = white_codes
    elif race == "hispanic":
        codes = hispanic_codes
        
    array = np.abs(array)
    
    ranking = {}
    ranking['tumsiz'] = array[-1]
    ranking['maligcount'] = array[-2]
    ranking['eod10_pn'] = array[-3]
    ranking['age_dx'] = array[-4]
    
    for key, val in codes.items():
        varname = key[1]
        start_idx = key[2]
        end_idx = start_idx + len(val)
        ranking[varname] = np.sum(array[start_idx:end_idx])
    
    d_view = [(name, score) for name,score in ranking.items()]
    d_view.sort(key=lambda x:x[1], reverse=True)
    for pair in d_view:
        print(pair)
    

## Decision Tree

In [ ]:

# min_sample_split: 300,400
# min_samples_leaf: 200
# max_depth: 130
# min_weight_fraction_leaf: .01
param_grid = [{'max_depth':[40,50,60], 'min_samples_leaf':[250,260,270,280,290]}]
tree_clf_reg = DecisionTreeClassifier()
dt_grid_search = GridSearchCV(tree_clf_reg, param_grid, cv=3, scoring=make_scorer(roc_auc_score), verbose=3)
dt_grid_search.fit(X_train, Y_train.astype(int))

In [ ]:
cvres = dt_grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)
print("Best: " + str(dt_grid_search.best_params_))

In [ ]:
results(dt_grid_search, X_test, Y_test)

In [ ]:
tree_clf_h = DecisionTreeClassifier(max_features=200, min_samples_leaf=150)
tree_clf_h.fit(X_train, Y_train)


In [ ]:
results(tree_clf_h, X_test, Y_test)

In [ ]:
feature_importance_rank(tree_clf_h.feature_importances_)

In [ ]:
sorted(enumerate(tree_clf_h.feature_importances_), key=lambda x:x[1], reverse=True)

## K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, Y_train.astype(int))

In [ ]:
results(knn_clf)

## Logistic Regression

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid = [{'C':[.01, .1, .75, 1, 1.5, 2]}]
lr_grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=3, 
                              scoring=make_scorer(roc_auc_score), verbose=5
                             )
lr_grid_search.fit(X_train, Y_train.astype(int))

In [ ]:
cvres = lr_grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)
print("Best: " + str(lr_grid_search.best_params_))

In [ ]:
results(lr_grid_search)

In [12]:
lr = LogisticRegression(C=1)
lr.fit(X_train, Y_train.astype(int))
results(lr)

Test accuracy score: 0.8514772424807027
ROC: 0.8593938519960583
PPV: 0.8810386473429952
NPV: 0.6314606741573033
Sensitivity: 0.9467878001297858
Specificity: 0.4162962962962963
G-Mean: 0.6278090908648487
Confusion matrix:
[[ 281  394]
 [ 164 2918]]


In [13]:
feature_importance_rank(lr.coef_[0])

('histo3v', 3.39341996171231)
('eod10_pn', 3.267788276115128)
('csexten', 2.8650025716839123)
('age_dx', 1.5197210849534828)
('csmetsdx', 1.1876860496228085)
('surgprif', 1.1161325031615117)
('no_surg', 0.5344834043744817)
('cslymphn', 0.4768705094045078)
('summ2k', 0.46698216460990444)
('tumsiz', 0.4576935487955035)
('reg', 0.4063436819838456)
('primsite', 0.3427083277427219)
('csrgeval', 0.3005766806999433)
('grade', 0.2894439613666429)
('maligcount', 0.28763796904697075)
('dx_conf', 0.27918579998842796)
('cstseval', 0.2098336004137446)
('mar_stat', 0.20503511365106583)
('beho3v', 0.1653866961214266)
('csmteval', 0.09928233744342337)
('sex', 0.08757370018726107)


In [ ]:
sorted(enumerate(lr.coef_[0]), key=lambda x:x[1], reverse=True)

## Random Forest

In [ ]:
#scoring=make_scorer(roc_auc_score)
param_grid = [{'max_features':[175,200], 'n_estimators':[10,15,20], 'min_samples_leaf':[150,175,200]}]
rf_grid_search = GridSearchCV(RandomForestClassifier(), 
            param_grid, cv=3, scoring=make_scorer(roc_auc_score), 
            verbose=5
)
rf_grid_search.fit(X_train, Y_train)

In [ ]:
cvres = rf_grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)
print("Best: " + str(rf_grid_search.best_params_))

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=20, min_samples_leaf=300, max_features=225, random_state=97)
rf_clf.fit(X_train, Y_train)
results(rf_clf)

In [ ]:
feature_importance_rank(rf_clf.feature_importances_)

In [ ]:
sorted(enumerate(rf_clf.feature_importances_), key=lambda x:x[1], reverse=True)

In [ ]:
results(rf_grid_search, X_test, Y_test)

## Linear SVM

In [ ]:
from sklearn.svm import LinearSVC
svm_clf = LinearSVC(C=.01)
svm_clf.fit(X_train, Y_train)

In [ ]:
results(svm_clf, X_test, Y_test)

In [ ]:
feature_importance_rank(svm_clf.coef_[0], "white")

In [ ]:
sorted(enumerate(svm_clf.coef_[0]), key=lambda x:x[1], reverse=True)

# Ensemble

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture
clf1 = RandomForestClassifier(n_estimators=20, min_samples_leaf=300, max_features=225)
clf2 = LogisticRegression(C=1)
clf3 = GaussianNB()
clf4 = GaussianMixture()

eclf = VotingClassifier(estimators=[
    ('rf', clf1), ('lr', clf2), ('nb', clf3), ('gm', clf4)
], voting='soft')
eclf.fit(X_train, Y_train)

In [ ]:
eclf.predict(X_test)

In [ ]:
results(eclf)

In [ ]:
gm_clf = GaussianMixture().fit(X_train, Y_train)

In [ ]:
gm_clf.predict(X_test)